<a href="https://colab.research.google.com/github/Vijith06/CDWInternShip/blob/day5/day_5_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bertviz

In [3]:
from transformers import pipeline, set_seed, AutoTokenizer, AutoModelForCausalLM, AutoModel
from torch import tensor, numel
from bertviz import model_view, head_view
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

set_seed(42)

In [4]:
# pipeline object in transformers provides easy access to transformer usage
MODEL = 'gpt2'

generator = pipeline('text-generation', model=MODEL)

# finish the sentence
generator("Hello, I'm a language model and I", max_length=30, num_return_sequences=3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model and I'll take you back to something I learned from a while ago in C++ classing. It's not"},
 {'generated_text': "Hello, I'm a language model and I didn't know this was possible. I'm now an eugene dell'arte (I guess"},
 {'generated_text': "Hello, I'm a language model and I'm a language model. My aim is to create the languages I would like to learn from our ancestors,"}]

In [5]:
# load up a tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

'Sinan' in tokenizer.get_vocab()

False

In [6]:
tokenizer.encode('Sinan loves a beautiful day')

[46200, 272, 10408, 257, 4950, 1110]

In [7]:
# encode a string and then convert the ids back into tokens. Note the Ġ character denoting a space before the token
tokenizer.convert_ids_to_tokens(tokenizer.encode('Sinan loves a beautiful day'))

['Sin', 'an', 'Ġloves', 'Ġa', 'Ġbeautiful', 'Ġday']

In [56]:
tokenizer.encode('good morning')  # ids

[11274, 3329]

In [59]:
encoded = tokenizer.encode('good morning', return_tensors='pt')  # as a pytorch tensor

encoded

tensor([[11274,  3329]])

In [58]:
# load up a tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForCausalLM.from_pretrained(MODEL)

In [60]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [61]:
encoded

tensor([[11274,  3329]])

In [62]:
model.transformer.wte(encoded)

tensor([[[ 0.0302, -0.0152,  0.0008,  ...,  0.0188, -0.2021, -0.0854],
         [ 0.0636,  0.1198,  0.0672,  ...,  0.0777,  0.0094, -0.1717]]],
       grad_fn=<EmbeddingBackward0>)

In [63]:
model.transformer.wte(encoded).shape  # 1 item in batch x 6 tokens x token dimension

torch.Size([1, 2, 768])

In [64]:
model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5]).reshape(1, 6)).shape  # manually create position vectors

torch.Size([1, 6, 768])

In [65]:
# create GPT input
initial_input = model.transformer.wte(encoded) + model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5]).reshape(1, 6))

initial_input.shape

RuntimeError: The size of tensor a (2) must match the size of tensor b (6) at non-singleton dimension 1

In [17]:
initial_input = model.transformer.drop(initial_input)  # run our input through the model's initual dropout later
initial_input

tensor([[[ 0.0107, -0.2453,  0.1275,  ..., -0.1969,  0.0006,  0.1539],
         [-0.1013, -0.0894, -0.0378,  ..., -0.0534, -0.0527,  0.0046],
         [-0.0716, -0.1690,  0.0386,  ..., -0.2034, -0.0197, -0.1113],
         [-0.0509, -0.0682,  0.1526,  ...,  0.0527,  0.0912, -0.0455],
         [ 0.0612,  0.1425,  0.1402,  ...,  0.0964,  0.0510,  0.1474],
         [-0.1283, -0.0632,  0.1287,  ..., -0.0907, -0.0655,  0.1085]]],
       grad_fn=<AddBackward0>)

In [18]:
model.lm_head

Linear(in_features=768, out_features=50257, bias=False)

In [19]:
for module in model.transformer.h:  # run the initial_input through every decoder in the stack
    initial_input = module(initial_input)[0]

initial_input = model.transformer.ln_f(initial_input)  # and then the final layer norm

In [20]:
initial_input

tensor([[[ 0.0542, -0.0179, -0.3388,  ..., -0.0948, -0.1067,  0.0129],
         [-0.4805,  0.1008, -0.7313,  ...,  0.0471, -0.4113,  0.0902],
         [ 0.0344, -0.2259, -0.5293,  ..., -0.1202,  0.1355,  0.2287],
         [-0.2374,  0.1787,  0.1845,  ..., -0.4057, -0.3617, -0.1861],
         [ 0.0235,  0.1212, -1.0182,  ..., -0.0597,  0.0020, -0.2220],
         [ 0.1206, -0.5034, -1.5260,  ..., -0.3367, -0.2821, -0.0410]]],
       grad_fn=<NativeLayerNormBackward0>)

In [21]:
# same as just running through the model
(initial_input == model(encoded, output_hidden_states=True).hidden_states[-1]).all()

tensor(True)

In [22]:
model(encoded).logits.shape

torch.Size([1, 6, 50257])

In [23]:
total_params = 0
for param in model.parameters():
    total_params += numel(param)

print(f'Number of params: {total_params:,}')

Number of params: 124,439,808


**Masked multi-headed attention**

In [24]:
import torch
import pandas as pd

In [25]:
phrase = 'My friend was right about this class. It is so fun!'
encoded_phrase = tokenizer(phrase, return_tensors='pt')

response = model(**encoded_phrase, output_attentions=True, output_hidden_states=True)

len(response.attentions)

`GPT2SdpaAttention` is used but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


12

In [26]:
encoded_phrase

{'input_ids': tensor([[3666, 1545,  373,  826,  546,  428, 1398,   13,  632,  318,  523, 1257,
            0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [27]:

response.attentions[-1].shape  # represtnations from the final decoder

torch.Size([1, 12, 13, 13])

In [28]:
encoded_phrase['input_ids'].shape

torch.Size([1, 13])

In [29]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])

tokens

['My',
 'Ġfriend',
 'Ġwas',
 'Ġright',
 'Ġabout',
 'Ġthis',
 'Ġclass',
 '.',
 'ĠIt',
 'Ġis',
 'Ġso',
 'Ġfun',
 '!']

In [30]:
# Layer index 9, head 0. Check out the almost 60% attention the token it is giving to the token class
arr = response.attentions[9][0][0]

n_digits = 3

attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)

attention_df.columns = tokens
attention_df.index = tokens

attention_df

<ipython-input-30-d6243f37e893>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)


,My,Ġfriend,Ġwas,Ġright,Ġabout,Ġthis,Ġclass,.,ĠIt,Ġis,Ġso,Ġfun,!
My,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġfriend,0.968,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġwas,0.824,0.145,0.031,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġright,0.979,0.008,0.007,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġabout,0.979,0.008,0.004,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġthis,0.924,0.031,0.007,0.006,0.016,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġclass,0.946,0.005,0.001,0.001,0.001,0.002,0.044,0.000,0.000,0.000,0.000,0.000,0.000
.,0.691,0.013,0.003,0.003,0.002,0.006,0.269,0.013,0.000,0.000,0.000,0.000,0.000
ĠIt,0.318,0.003,0.003,0.003,0.006,0.018,0.599,0.018,0.032,0.000,0.000,0.000,0.000
Ġis,0.331,0.006,0.002,0.002,0.003,0.018,0.533,0.013,0.062,0.030,0.000,0.000,0.000


In [31]:
head_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [32]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
model_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [33]:
response.hidden_states[-1].shape

torch.Size([1, 13, 768])

In [34]:
response.logits

tensor([[[ -33.0735,  -32.3348,  -35.2379,  ...,  -38.3576,  -38.4758,
           -33.0943],
         [ -98.8075,  -98.9730, -105.2267,  ..., -109.0421, -104.7381,
          -101.0802],
         [-131.2765, -130.8177, -135.2054,  ..., -138.3585, -134.9846,
          -134.1657],
         ...,
         [-120.6479, -122.7569, -127.1870,  ..., -130.4343, -129.5816,
          -125.1274],
         [ -73.8157,  -78.6168,  -85.3994,  ...,  -91.8286,  -91.7845,
           -81.4570],
         [-134.7547, -134.9495, -136.2912,  ..., -147.1823, -148.1732,
          -130.7045]]], grad_fn=<UnsafeViewBackward0>)

In [35]:
response.logits.shape

torch.Size([1, 13, 50257])

In [36]:
# look at the top next token in the auto-regressive language modelling task
pd.DataFrame(
    zip(tokens, tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])),
    columns=['Sequence up until', 'Next token with highest probability']
)

,Sequence up until,Next token with highest probability
0,My,Ċ
1,Ġfriend,","
2,Ġwas,Ġa
3,Ġright,.
4,Ġabout,Ġthat
5,Ġthis,.
6,Ġclass,.
7,.,ĠI
8,ĠIt,'s
9,Ġis,Ġa


In [37]:
generator('My friend was right', max_length=12, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "My friend was right - it doesn't matter how hard she"},
 {'generated_text': 'My friend was right," his partner said. "I think'},
 {'generated_text': "My friend was right when: he said that they don't"},
 {'generated_text': 'My friend was right when I said that there were two options'},
 {'generated_text': 'My friend was right when I told him.\n\n"'}]

In [38]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=False)  # greedy search

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this class. It is so fun! I love it! I love the'}]

In [39]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=True)  # greedy search with sampling

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this class. It is so fun! My 3 year old was playing on'}]

**Parameters**

In [40]:
from transformers import AutoTokenizer

# Use the id of the model you want to use
# GPT-2 "openai-community/gpt2"
# Qwen "Qwen/Qwen2-0.5B"
# SmolLM "HuggingFaceTB/SmolLM-135M"

prompt = "It was a dark and stormy"
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
input_ids = tokenizer(prompt).input_ids
input_ids

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[1026, 373, 257, 3223, 290, 6388, 88]

In [41]:
for t in input_ids:
    print(t, "\t:", tokenizer.decode(t))

1026 	: It
373 	:  was
257 	:  a
3223 	:  dark
290 	:  and
6388 	:  storm
88 	: y


**Prediction - Top_K**

In [42]:
# We tokenize again but specifying the tokenizer that we want it to
# return a PyTorch tensor, which is what the model expects,
# rather than a list of integers
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model(input_ids)
outputs.logits.shape  # An output for each input token

torch.Size([1, 7, 50257])

In [43]:
final_logits = model(input_ids).logits[0, -1]  # The last set of logits
final_logits.argmax()  # The position of the maximum

tensor(1755)

In [44]:
tokenizer.decode(final_logits.argmax())

' night'

In [45]:
import torch

top10_logits = torch.topk(final_logits, 10)
for index in top10_logits.indices:
    print(tokenizer.decode(index))

 night
 day
 evening
 morning
 afternoon
 summer
 time
 winter
 weekend
,


In [46]:
top10 = torch.topk(final_logits.softmax(dim=0), 10)
for value, index in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(index):<10} {value.item():.2%}")

 night     46.18%
 day       23.46%
 evening   5.87%
 morning   4.42%
 afternoon 4.11%
 summer    1.34%
 time      1.33%
 winter    1.22%
 weekend   0.39%
,          0.38%


**Max Tokens**

In [47]:
output_ids = model.generate(input_ids, max_new_tokens=20)
decoded_text = tokenizer.decode(output_ids[0])

print("Input IDs", input_ids[0])
print("Output IDs", output_ids)
print(f"Generated text: {decoded_text}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Input IDs tensor([1026,  373,  257, 3223,  290, 6388,   88])
Output IDs tensor([[ 1026,   373,   257,  3223,   290,  6388,    88,  1755,    13,   383,
          2344,   373, 19280,    11,   290,   262, 15114,   547,  7463,    13,
           383,  2344,   373, 19280,    11,   290,   262]])
Generated text: It was a dark and stormy night. The wind was blowing, and the clouds were falling. The wind was blowing, and the


**Beams**

In [48]:
beam_output = model.generate(
    input_ids,
    num_beams=5,
    max_new_tokens=30,
)

print(tokenizer.decode(beam_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It was a dark and stormy night.

"It was dark and stormy," he said.

"It was dark and stormy," he said.




In [49]:
beam_output = model.generate(
    input_ids,
    num_beams=5,
    repetition_penalty=2.0,
    max_new_tokens=38,
)

print(tokenizer.decode(beam_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It was a dark and stormy night.

"I'm not sure what happened, but I don't know if it was an accident or something," he said.

Police say they're still trying to determine


**Do Sample**

In [50]:
from transformers import set_seed

# Setting the seed ensures we get the same results every time we run this code
set_seed(70)

sampling_output = model.generate(
    input_ids,
    do_sample=True,
    max_new_tokens=34,
    top_k=0,  # We'll come back to this parameter
)

print(tokenizer.decode(sampling_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It was a dark and stormy day until it broke down the big canvas on my sleep station, making me money dilapidated, and, with a big soothing mug, I surrendered to the rolu


**Temperature**

In [51]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.4,
    max_new_tokens=40,
    top_k=0,
)

print(tokenizer.decode(sampling_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It was a dark and stormy night in the city, and the sky was dark and dark. It wasgood for the sun to shine. It was a day of joy. It was a day of joy Rhino was in the middle


In [52]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.001,
    max_new_tokens=40,
    top_k=0,
)

print(tokenizer.decode(sampling_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It was a dark and stormy night. The wind was blowing, and the clouds were falling. The wind was blowing, and the clouds were falling. The wind was blowing, and the clouds were falling. The wind was blowing,


In [53]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=3.0,
    max_new_tokens=40,
    top_k=0,
)

print(tokenizer.decode(sampling_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It was a dark and stormy time Robinson lower display magn Simon launching warmth cube lightsaber pil Welshatoryfaith317build thirty Lucy mixShot disapproval newplings Seven presentation laced & laced Einstein award drug camouflage 451 dietary downloading participants / pay Power delegated


**Top-K**

In [54]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    max_new_tokens=40,
    top_k=5,
)

print(tokenizer.decode(sampling_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It was a dark and stormy evening. It was a time of mourning, of mourning for those who had lost loved ones, of mourning for the loss of those who had been murdered. It was a time of mourning, a time


**Top - P**

In [55]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    max_new_tokens=40,
    top_p=0.94,
    top_k=0,
)

print(tokenizer.decode(sampling_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It was a dark and stormy day and some people were not there - I was hoping for some sort of action. I didn't see the fire in the sky but had to leave the location to get to the chapel as I was
